In [49]:
import tensorflow as tf
from tensorflow.python.keras import Model, Input
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras.optimizers import Adadelta
from tensorflow.python.keras.estimator import model_to_estimator

from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np

%matplotlib inline

In [50]:
NUM_CLASSES = 7
IMAGE_SHAPE = (112, 112, 3)

In [51]:
def parse(serialized):
    # Define a dict with the schema reflecting the data in the TFRecords file
    features = \
        {
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        }
    
    # Parse the serialized data
    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)
    
    # Get the image as raw bytes
    image_raw = parsed_example['image']
    
    # Convert the raw bytes to tensorflow datatypes
    image = tf.decode_raw(image_raw, tf.uint8)
    image = tf.cast(image, tf.float32)
    image = image/255
    image = tf.reshape(image, IMAGE_SHAPE)
    print(image)
    
    # Get the label
    label = parsed_example['label']
    label = tf.one_hot(label, NUM_CLASSES)
    
    # Return the image and label as correct data types
    return image, label
    

In [52]:
def input_fn(filenames, train, batch_size=32, buffer_size=2048):
    # Create a TensorFlow Dataset-object which has functionality for reading and shuffling data 
    # from TFREcords files
    
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    
    # Start building the pipeline
    # Parse
    dataset = dataset.map(parse)
    
    # Shuffle when training
    if train:
        dataset = dataset.shuffle(buffer_size = buffer_size)
        # Allow infinite reading of the data
        num_repeat = None
    else:
        num_repeat = 1
        
    # Repeat the dataset the given number of times
    dataset = dataset.repeat(num_repeat)
    
    # Set the batch size
    dataset = dataset.batch(batch_size)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    images_batch, labels_batch = iterator.get_next()
    
    return {'image':images_batch}, labels_batch
    
    

In [53]:
filenames = '../data/tfrecords/training.tfrecords'

dataset = tf.data.TFRecordDataset(filenames=filenames)   
# Start building the pipeline
# Parse
dataset = dataset.map(parse)
     
# Repeat the dataset the given number of times
dataset = dataset.repeat(1)
    
    # Set the batch size
dataset = dataset.batch(2)
    
    # Create an iteratori
iterator = dataset.make_one_shot_iterator()
    
images_batch, labels_batch = iterator.get_next()

with tf.Session() as sess:
    for i in range(1):
        value = sess.run(labels_batch)
        print(value)
    



Tensor("Reshape:0", shape=(112, 112, 3), dtype=float32)
[[0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]


In [54]:
path_tfrecords_train = '../data/tfrecords/training.tfrecords'
def train_input_fn():
    return input_fn(filenames=path_tfrecords_train, train=True)

In [55]:
path_tfrecords_test = '../data/tfrecords/validation.tfrecords'
def test_input_fn():
    return input_fn(filenames=path_tfrecords_test, train=False)

In [56]:
image1 = np.array(Image.open('../data/validation/st1179_sound_9.PNG'))
image2 = np.array(Image.open('../data/validation/st1179_dry_knot_3.PNG'))
some_images = np.array([image1, image2])/255
some_images = np.reshape(some_images, (2, 37632))


In [57]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": some_images.astype(np.float32)},
    num_epochs=1,
    shuffle=False)

In [62]:



inputs = Input(shape=(112, 112, 3), name='image')
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
y = Dense(7, activation='softmax')(x)

keras_model = Model(inputs=inputs, outputs=y)

keras_model.compile(optimizer = Adadelta(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])



In [63]:
keras_estimator = model_to_estimator(keras_model = keras_model, model_dir='../checkpoints')

INFO:tensorflow:Using the Keras model from memory.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3bfc066208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [66]:
keras_estimator.train(input_fn=train_input_fn, steps=8000)


Tensor("Reshape:0", shape=(112, 112, 3), dtype=float32)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


ResourceExhaustedError: OOM when allocating tensor with shape[186624,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adadelta/Variable_12/Assign = Assign[T=DT_FLOAT, _class=["loc:@training/Adadelta/Variable_12"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adadelta/Variable_12, training/Adadelta/zeros_4)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Adadelta/Variable_12/Assign', defined at:
  File "/home/demouser/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/demouser/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/demouser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_

In [ ]:
result = keras_estimator.evaluate(input_fn=test_input_fn)

In [ ]:
predictions = keras_estimator.predict(input_fn=predict_input_fn)

In [ ]:
cls_pred = np.array(list(predictions))
cls_pred